# Algorithm 4: Relative Position Encoding (relpos)

Relative position encoding adds positional information to the pair representation by encoding the distance between residue positions.

## Algorithm Pseudocode

![relpos](../imgs/algorithms/relpos.png)

## Source Code Location
- **File**: `AF2-source-code/model/modules.py`
- **Location**: `EmbeddingsAndEvoformer.__call__`
- **Lines**: 1744-1758

In [ ]:
import numpy as np

np.random.seed(42)

In [ ]:
def one_hot(indices, num_classes):
    """Algorithm 5: One-hot encoding."""
    return np.eye(num_classes)[indices]


def relpos(residue_index, max_relative_feature=32):
    """
    Relative Position Encoding - Algorithm 4.
    
    Computes relative position features between all residue pairs.
    
    Args:
        residue_index: [N_res] residue position indices
        max_relative_feature: Maximum relative distance to encode
    
    Returns:
        rel_pos: [N_res, N_res, 2*max+1] one-hot encoded relative positions
    """
    N_res = len(residue_index)
    
    # Step 1: Compute relative positions (Line 1)
    # offset[i,j] = residue_index[i] - residue_index[j]
    offset = residue_index[:, None] - residue_index[None, :]
    
    print(f"Relative position offset: {offset.shape}")
    print(f"Offset range: [{offset.min()}, {offset.max()}]")
    
    # Step 2: Clip to [-max, max] (Line 2)
    clipped = np.clip(offset, -max_relative_feature, max_relative_feature)
    
    # Step 3: Shift to [0, 2*max] for one-hot encoding (Line 3)
    shifted = clipped + max_relative_feature
    
    # Step 4: One-hot encode (Algorithm 5)
    num_classes = 2 * max_relative_feature + 1
    rel_pos = one_hot(shifted.astype(int), num_classes)
    
    print(f"Output shape: {rel_pos.shape}")
    print(f"Number of relative position classes: {num_classes}")
    
    return rel_pos

In [ ]:
# Test
N_res = 32
residue_index = np.arange(N_res)  # Standard sequential indices

print("Test Relative Position Encoding")
print("="*50)

rel_pos = relpos(residue_index, max_relative_feature=32)

# Verify structure
print(f"\nVerification:")
print(f"  Position (0,0): relative pos = 0, one-hot index = 32")
print(f"  Actual one-hot peak: {np.argmax(rel_pos[0, 0])}")
print(f"  Position (0,5): relative pos = -5, one-hot index = 27")
print(f"  Actual one-hot peak: {np.argmax(rel_pos[0, 5])}")

## Source Code Reference

```python
# From AF2-source-code/model/modules.py

# Relative position encoding.
# Jumper et al. (2021) Suppl. Alg. 4 "relpos"
# Jumper et al. (2021) Suppl. Alg. 5 "one_hot"
if c.max_relative_feature:
  pos = batch['residue_index']
  offset = pos[:, None] - pos[None, :]
  rel_pos = jax.nn.one_hot(
      jnp.clip(
          offset + c.max_relative_feature,
          a_min=0,
          a_max=2 * c.max_relative_feature),
      2 * c.max_relative_feature + 1)
  pair_activations += common_modules.Linear(
      c.pair_channel, name='pair_activiations')(rel_pos)
```